# Exercise 3 - Data Oriented Programming Paradigms - Group 20

## Team Members: Schöbinger-Hassve Sebastian, Kurteshi Mal, Kaçuri Muhamet, Ademi Ard

#### Topic: What are typical characteristics for a large flow of refugees? Are there typical characteristics of large flows of immigration? Can refugee flows between two countries be predicted?

In [1]:
import numpy as np
import seaborn
import pandas as pd
import matplotlib.pyplot as plt
import pycountry as pc
import math
import seaborn as sns
from IPython.core.display import display, HTML
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, BayesianRidge, SGDRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## Which dataset(s) did you choose? Why?

The dataset picked in this exercise are the once listed below: 
- UN List of Refugees + Venezuelans displaced abroad (Link: https://www.unhcr.org/refugee-statistics/download/?url=L6zuyL)
- Worldwide Governance Indicators (https://datacatalog.worldbank.org/dataset/worldwide-governance-indicators)
- GDP(Gross domestic product) for Countries throw years (https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=# ) 
- UCDP One-sided Violence Dataset version 20.1 (https://ucdp.uu.se/downloads/index.html#onesided) 
- Death by natural disasters (https://ourworldindata.org/grapher/natural-disaster-deaths-ihme?tab=table)
- Conflicts 
- 10 Distances (https://www.geodatos.net/en/distances/countries/)

We have choosen these datasets in order to be able to obtain as good insights as possible for our aim to answer these questions. As the base dataset we have chosen the UN list of refugees and we have added additional information to this dataset by merging the data from other datasets the we thought would be helpful. We think that some factors like governance indicators, GDP, violence, conflicts and natural disastres can have an impact to the immigration flow, mainly this is the reason why we have chosen to enrich our base dataset with this additional information. Another information that will be helpful in our further analysis is the distance between the country of origin and the country of asylum, since this information is absent in the UN list of refugees we have chosesn to merge this information from another dataset that contains these distances.


## How did you clean/transform the data? Why?

Our merged dataset contains a lot of missing information since we have added information from different sources. In the cases where the information was missing and we couldn’t jus impute these missing values like for example in the most obvious case there are years that we simply don’t have information for a lot of features that we added from different sources.
First, we had to chose a time span between 1996 and 2018 since not all datasets covered the same years. We did some manual correction of the data like adding some missing ISO values. Another cleaning decision that we made was to remove stateless refugees since we thought that this information would not be useful in the answers that we want to get from the data. We noticed that some country names needed to be transformed since they included ‘,’ and this messed with csv format. We also did some manual imputation on the distances between countries since some distances weren’t covered in the distance dataset.
The GDP values are joined with the "Control of Corruption", "Rule of Law" estimators into one dataset. Once the overall dataset is created, missing values are dealt with: 1. in case of a few missing columns, they are imputed with the estimated difference of previous/later values 2, if the row is entirely empty it is filled up with 0 and an additional column is set to 1, indicating that the values of the statistics attributes were NaNs.

### Initialize Dataframes

In [2]:
gdp_df = pd.read_csv('./1_data_acquisition/dataset_03/gdp.csv', sep=',')
distance_df = pd.read_csv('./1_data_acquisition/dataset_10.csv', sep=',')
refugee_df = pd.read_csv('./1_data_acquisition/dataset_01/fixed_population.csv', sep=',')
statistics_df = pd.read_csv('./1_data_acquisition/dataset_02/WGIData.csv', sep=',')
natural_death_df = pd.read_csv('./1_data_acquisition/dataset_07/natural-disaster-deaths-ihme.csv', sep=',')
conflict_df = pd.read_csv('./1_data_acquisition/dataset_06/ucdp-onesided-201.csv', sep=',')

### Reduce dataset to usable years
Since not all datasets cover the same time period, we settled for a time span between 1996 and 2018. We chose against covering also 2019, since multiple countries lacked their GDP for 2019 in the responsible dataset.

In [3]:
start_year = 1996
end_year = 2018

refugee_df = refugee_df[(refugee_df['Year'] >= start_year) & (refugee_df['Year'] <= end_year)]

### Data manipulation 

In the section below we are going to define functions that will help us for the data tranformation in features like distance in between coutries. This action is nesscesary because of the need for a defined distance in between country of origin and the country of destination. The distance is presented in miles and from the functions below is made uniform for one merged dataframe in future. 

In [4]:
def get_short(entry):
    val = pc.countries.get(name=entry['Country Name'])
    if not val:
        val = pc.countries.get(common_name=entry['Country Name'])
        
    if not val:
        try:
            vals = pc.countries.search_fuzzy(entry['Country Name'])
            if len(vals) == 1:
                val = vals[0]
        except:
            return ""
    
    if val:
        return val.alpha_3
    else:
        return ""    
    
def add_distance(entry):
    src_country = entry['Country of origin (ISO)']
    dst_country = entry['Country of asylum (ISO)']
    
    src_name = country_lookup[country_lookup['ISO'] == src_country]['Country Name'].values[0]
    dst_name = country_lookup[country_lookup['ISO'] == dst_country]['Country Name'].values[0]
    
    distance_entry = distance_df[(distance_df['Source Country Norm'] == src_name) &
                                 (distance_df['Destination Country Norm'] == dst_name)]
    
    dist_val = distance_entry['Shortest distance'].values
    
    if len(dist_val) == 0 or np.isnan(dist_val):
        dist_val = distance_entry['Shortest distance between major cities'].values
        
    if len(dist_val) == 0 or np.isnan(dist_val):
        return np.nan
    else:
        return dist_val[0]
    
def update_iso(country_name, iso):
    ix =country_lookup[country_lookup['Country Name'] == country_name].index
    country_lookup.loc[ix,'ISO'] = iso
    
def update_country(src, dst, val):
    ix = distance_df[(distance_df['Source Country Norm']== src) &
                     (distance_df['Destination Country Norm']== dst)].index
    distance_df.loc[ix, 'Shortest distance'] = val
    
    ix = distance_df[(distance_df['Source Country Norm']== dst) & 
                     (distance_df['Destination Country Norm']== src)].index
    distance_df.loc[ix, 'Shortest distance'] = val

### Setup Country lookup Table

Create a dataframe, from which we can lookup the Name and corresponding ISO 3166-1 alpha-3 format

In [5]:
country_lookup = pd.DataFrame(columns=['Country Name', 'ISO'])
country_lookup['Country Name'] = np.unique(distance_df['Source Country Norm'].values)
country_lookup['ISO'] = country_lookup.apply(lambda x: get_short(x), axis=1)
update_iso("Kosovo", "XKX")
update_iso("Curacao", "CUW")
update_iso("Iran", "IRN")
update_iso("South Korea", "KOR")
update_iso("Bonaire, Saint Eustatius and Saba ", "BES")
update_iso("Cocos Islands", "CCK")
update_iso("Democratic Republic of the Congo", "COD")
update_iso("Laos", "LAO")
update_iso("Netherlands Antilles", "NLD")
update_iso("Palestinian Territory", "PSE")
update_iso("Timor Leste", "TLS")
update_iso("U.S. Virgin Islands", "VIR")
update_iso("North Korea", "PRK")
update_iso("Ivory Coast", "CIV")
update_iso("Reunion", "REU")
update_iso("Sint Maarten", "SXM")

### Clean Refugee Dataset

* Remove stateless refugees
* Add "Venezuelans abroad" to the refugee counter
* Manually modify country ISO connections

In [6]:
# Add venezuelan refugees to overall count
refugee_df['Venezuelans displaced abroad'].fillna(0, inplace=True)
refugee_df['Refugees under UNHCR’s mandate'] = refugee_df['Refugees under UNHCR’s mandate'] + refugee_df['Venezuelans displaced abroad']
refugee_df.drop(['Venezuelans displaced abroad'], inplace=True, axis=1)


# Drop all refugees where we don't know where they come from
refugee_df.dropna(subset=['Country of origin (ISO)', 'Country of asylum (ISO)'], inplace=True)
refugee_df = refugee_df[refugee_df['Country of origin (ISO)'] != "XXA"]
refugee_df = refugee_df[refugee_df['Country of asylum (ISO)'] != "XXA"]

# Make Tibet part of China
ix = refugee_df[refugee_df['Country of origin (ISO)'] == "TIB"].index
refugee_df.loc[ix,'Country of origin (ISO)'] = "CHN"

ix = refugee_df[refugee_df['Country of asylum (ISO)'] == "TIB"].index
refugee_df.loc[ix,'Country of asylum (ISO)'] = "CHN"

# Remove , which mess with the csv format
refugee_df['Country of origin'] = refugee_df['Country of origin'].str.replace(',', '.')
refugee_df['Country of asylum'] = refugee_df['Country of asylum'].str.replace(',', '.')

### Add Distances to refugee Dataset

Manually add distances and add the distance between the country of origin and country of asylum

In [ ]:
update_country('China', 'India', 0)
update_country('Venezuela', 'Curacao', 0)
update_country('Venezuela', 'Aruba', 0)
update_country('Liberia', 'Nigeria', 1138)
update_country('Rwanda', 'Burundi', 0)
update_country('Tunisia', 'Austria', 1052)
update_country('Nicaragua', 'Spain', 8376)
update_country('Sudan', 'Liberia', 3328)
update_country('Iran', 'Spain', 3445)
update_country('Kyrgyzstan', 'Spain', 5433)
update_country('Belarus', 'Sweden', 911)
update_country('Nicaragua', 'Spain', 8376)
update_country('Sudan', 'Liberia', 3328)
update_country('Iran', 'Spain', 3445)
update_country('Kyrgyzstan', 'Spain', 5433)
update_country('Belarus', 'Sweden', 911)
update_country('Nicaragua', 'Belgium', 8424)
update_country('Nicaragua', 'Italy', 8825)
update_country('Benin', 'Netherlands', 4288)
update_country('Bangladesh', 'Trinidad and Tobago', 14730)
update_country('Andorra', 'Germany', 1170)
update_country('Niger', 'United Kingdom', 3187)
update_country('Andorra', 'United States', 7864)
update_country('Eritrea', 'Bosnia and Herzegovina', 3812)
update_country('Niger', 'Germany', 1170)
update_country('Jordan', 'Hungary', 1963)
update_country('Malaysia', 'United Kingdom', 9999)
update_country('Sudan', 'Iceland', 5538)

print('Inserting Distances')
refugee_df.insert(refugee_df.shape[1], 'Distance', refugee_df.apply(lambda x: add_distance(x), axis=1))

Inserting Distances


#### Support Functions

In [ ]:
# Handling empty entries ('..') from the original csv file
def gdp_fix(entry, country):
    
    year = entry['Year']
    
    src = entry['{} (ISO)'.format(country)]
    
    gdp_entry = gdp_df[(gdp_df['Country Code'] == src)]
    gdp = gdp_entry['{} [YR{}]'.format(year, year)].values
    
    if len(gdp_entry) == 0 or gdp[0] == "..":
        return np.nan
            
    return gdp[0]

def entry_hole_plugger(entry, min_year, max_year):

    first_year = -1
    first_val = -1
    last_year = -1
    last_val = -1
    empty_years = []
    
    for i in range(min_year, max_year):
        field_name = '{}'.format(i)
            
        if field_name in entry:
            year_value = entry[field_name]
        else:
            year_value = ''
        if year_value == '' or year_value == '..':
            if first_year != -1:
                empty_years.append(i)    
            continue  
        if len(empty_years) == 0:
            first_year = i
            first_val = float(year_value)
        else:
            last_year = i
            last_val = float(year_value)
            
        # Fill up empty years between two known years        
        if first_year+1 < last_year: 
            val_diff = last_val - first_val  
            years_diff = last_year - first_year             
            yearly_diff = val_diff/years_diff 
            for empty_year in empty_years:
                now_years = empty_year - first_year 
                now_val = first_val + (now_years * yearly_diff) 
                now_val = round(now_val, 5)                
                field_name = '{}'.format(empty_year)                    
                entry[field_name] = str(now_val)
                
            first_year = i
            first_val = float(year_value)
            empty_years = []
            
    # Fill up empty first or last years
    for i in range(min_year, max_year):
        field_name = '{}'.format(i)
            
        if field_name in entry:
            year_value = entry[field_name]
        else:
            year_value = ''
            
                
        if year_value == '' or year_value == '..':
            if first_year == -1:
                empty_years.append(i)
            
            continue
            
        first_year = last_year
        first_val = last_val
        
        last_year = i
        last_val = float(year_value)
                  
        if last_year == first_year + 1 and len(empty_years) > 0:
            val_diff = last_val - first_val
            for empty_year in empty_years:
                now_years = first_year - empty_year 
                now_val = first_val + (now_years * val_diff) 
                now_val = round(now_val, 5)
                field_name = '{}'.format(empty_year)         
                entry[field_name] = str(now_val)
                
            empty_years = []
            
    
    if len(empty_years) > 0:
        val_diff = last_val - first_val
        for empty_year in empty_years:
            now_years = first_year - empty_year 
            now_val = first_val + (now_years * val_diff) 
            now_val = round(now_val, 5)
            field_name = '{}'.format(empty_year)         
            entry[field_name] = str(now_val)
    
    entry = entry.sort_index()
                    
    return entry

def add_stats_column(entry, columns, country):
    year = entry['Year']
    src = entry['{} (ISO)'.format(country)]
    
    stat_entry = overall_df[overall_df['Country'] == src][year]
    values = stat_entry.values.tolist()

    if len(values) != 7:
        print(src)
        print(stat_entry)
        values = values[:7]
        print(len(values))
    
    return values

def add_natural_death(entry):
    natural_death_entry = natural_death_df[(natural_death_df['Year'] == entry['Year']) &
                                           (natural_death_df['Code'] == entry['Country of origin (ISO)'])]
    if len(natural_death_entry) == 1:
        value = natural_death_entry['Deaths - Exposure to forces of nature - Sex: Both - Age: All Ages (Number)'].values
        return value[0]
    else:
        return 0
    
def add_value_is_na(entry, columns):
    nas = entry[columns].isna().any()
    if nas:
        return 1
    else:
        return 0

### Country Statistics

The GDP values are joined with the "Control of Corruption", "Rule of Law", .. estimators into one dataset.
Once the overall dataset is created, missing values are dealt with:
1. in case of a few missing columns, they are imputed with the estimated difference of previous/later values
2. if the row is entirely empty, it is filled up with 0 and an additional column is set to 1, indicating that the values of the statistics attributes were NaNs

In [ ]:
gdp_year_columns = {}
all_year_columns = [str(i) for i in range(1960, 2019)]
stats_year_columns = [str(i) for i in range(start_year, end_year+1)]

for i in range(1960, 2021):
    gdp_year_columns['{} [YR{}]'.format(i, i)] = str(i)
    
gdp_df = gdp_df.rename(gdp_year_columns, axis=1)    
gdp_df[all_year_columns] = gdp_df[all_year_columns].apply(lambda x: entry_hole_plugger(x, 1960, 2019), axis=1)

statistics_columns = ["Control of Corruption: Estimate", 
                      "Government Effectiveness: Estimate", 
                      "Political Stability and Absence of Violence/Terrorism: Estimate", 
                      "Regulatory Quality: Estimate",
                      "Rule of Law: Estimate",
                      "Voice and Accountability: Estimate"]

overall_columns = ['GDP']
overall_columns += statistics_columns        

values = []
countries = np.unique(list(country_lookup['ISO'].values))
for country in countries:
    # Add GPD Values
    tmp_list = gdp_df[gdp_df['Country Code'] == country].get(stats_year_columns).values.tolist()
    if len(tmp_list) == 0:
        tmp_list = [np.nan for i in range(start_year, end_year+1)]
    else:
        tmp_list = tmp_list[0]
        tmp_list = list(map(lambda x: np.nan if x == ".." or x == "" else x, tmp_list))
        
    tmp_list = [country, 'GDP'] + tmp_list
        
    values.append(tmp_list)
    
    country_df = statistics_df[statistics_df['Country Code'] == country]
    
    for column in statistics_columns:
        tmp_df = country_df[country_df['Indicator Name'] == column].filter(items=stats_year_columns)
        if len(tmp_df) == 0:
            tmp_list = [np.nan for i in range(start_year, end_year+1)]
        else:
            tmp_df = tmp_df.apply(lambda x: entry_hole_plugger(x, start_year, end_year+1), axis=1)
            tmp_list = tmp_df.get(stats_year_columns).values.tolist()[0]

        tmp_list = [country, column] + tmp_list    
        
        values.append(tmp_list)
    
column_names = ['Country', 'Statistics']
column_names +=  [i for i in range(start_year, end_year+1)]

overall_df = pd.DataFrame(data=values, columns=column_names)

In [ ]:
columns = ["GDP",
           "Control of Corruption: Estimate", 
           "Government Effectiveness: Estimate", 
           "Political Stability and Absence of Violence/Terrorism: Estimate", 
           "Regulatory Quality: Estimate",
           "Rule of Law: Estimate",
           "Voice and Accountability: Estimate"]

country_cols = {'Country of origin': '_origin', 'Country of asylum': '_asylum'}

column_names = []

for country, suffix in country_cols.items():
    column_names = []
    for column in columns: 
        if ':' in column:
            column_name, _ = column.split(':', 1)
        else:
            column_name = column
        
        column_names.append('{}{}'.format(column_name, suffix))
                
    print('Working on {}'.format(suffix))
    content = refugee_df.apply(lambda x: add_stats_column(x, columns, country), axis=1)
    for i in range(0, 7):
        val = []
        for e in content:
            val.append(e[i])
            
        refugee_df[column_names[i]] = val
        
    refugee_df['isna_GDP{}'.format(suffix)] = refugee_df.apply(lambda x: add_value_is_na(x, ['GDP{}'.format(suffix)]), axis=1)
    refugee_df['isna_statistics{}'.format(suffix)] = refugee_df.apply(lambda x: add_value_is_na(x, column_names[1:]), axis=1)
    
refugee_df = refugee_df.fillna(0)

### Add "deaths due to nature" to origin country

The amount of deaths which were caused by natural incidents (storms, earthquake, ...)

In [ ]:
refugee_df.insert(refugee_df.shape[1], 'Natural Deaths in origin', refugee_df.apply(lambda x: add_natural_death(x), axis=1))

### Add conflict status to origin country

Add the conflict indicator to the country of origin. This indicates (civil) wars or other types of unrests which might have caused the refugees to flee

The civil wars in Tijikistan and the Ukraine were additionally added

In [ ]:
def joinable(entry):
    question = pair_conflict_df[(pair_conflict_df['Year'] == entry['Year']) &
                               (pair_conflict_df['ISO'] == entry['Country of origin (ISO)'])]
    if len(question) > 0:
        return 1
    else:
        return 0
    
def show_conflict(entry):
    
    for country in entry['location'].split(','):
        country = country.strip()
        country = country.split('(', 1)[0].strip()
        if country == "DR Congo":
            country = "Democratic Republic of the Congo"
        elif country == "Congo":
            country = "Republic of the Congo"
        elif country == "Bosnia-Herzegovina":
            country = "Bosnia"
        alpha = country_lookup[country_lookup['Country Name'] == country]['ISO'].values
        if len(alpha) == 0:
            try:
                countries = pc.countries.search_fuzzy(country)
                if len(countries) > 1:
                    print(country)
                    raise Exception
                return entry['year'], countries[0].alpha_3
            except:
                print('We couldn\'t find {}'.format(country))
                return country
        
        return entry['year'], alpha[0]

In [ ]:
conflict_data = conflict_df.apply(lambda x: show_conflict(x), axis=1)
pair_conflict_df = pd.DataFrame.from_records(conflict_data, columns=['Year', 'ISO'])

# Adding civil war in Tijikistan
for year in range(1992, 1998):
    pair_conflict_df = pair_conflict_df.append({'Year': year, 'ISO':'TJK'}, ignore_index=True)
    
for year in range(2014, 2019):
    pair_conflict_df = pair_conflict_df.append({'Year': year, 'ISO':'UKR'}, ignore_index=True)

refugee_df['Conflict in origin'] = refugee_df.apply(lambda x: joinable(x), axis=1)

### Display Ratios and Save to file

Check the final dataset for NaN values

In [ ]:
def get_ratio(my_df):
    print(len(my_df))
    print(round(my_df.isna().sum() / len(my_df) * 100, 2))
    
get_ratio(refugee_df)

### Save final dataset to file

In [ ]:
refugee_df.to_csv('./2_dataset/final_dataset_preprocesed_v6.csv', index=False)

## How did you solve the problem of missing values? Why?

After the data transformation we ended still with missing values in our final dataset. 

The first approach included using KNNImputer
The reason why we thought this would be useful is basen on the way on how this algorithm works in which each sample’s missing values are imputed using the mean value from n_neighbors nearest neighbors found in the training set. We thought that these missing values would be inputted with values that would be at least close to real values.

The second attempt following approach was used:

(V = existing values, X = missing values)
Case 1: V V V X X X V V
Case 2: X X X V V V X X
Case 3: X X X X X X X X

For Case 1, the missing values X were filled up with the difference of two existing values next to the missing ones.
For Case 2, the missing values X were filled up with the difference of the two consecutive values prior or after.
For Case 3, the missing values X were replaced with 0 and an additional column was set to 1, indicating that this row contained NaN values which could not be replaced.

See function "entry_hole_plugger" and "add_value_is_na"

## What questions did you ask of the data? Why were these good questions?

The questions asked of the data are:
- 1. What characteristics cause the refugee flows?
- 2. Do some events (i.e. war) initiate a higher amount of refugees than others (i.e. natural disasters, economic breakdown)?
- 3. What is the connection between origin and destination country? 

These questions were in very interest to us because they lead important insights to us such as indication on why the migration happens, what can be the reasons, where the migration starts, is there any correlation of the origin contry and the neighbourhood countries. What are factors and indications that impact the number of refugees etc.  


## What were the answers to these questions? How did you obtain them? Do the answers make sense?

- 1. What characteristics cause the refugee flows?

The answer to this question is based on the relation of number of refugees with the GDP value of the origin contry of migration, with the controll of corruption level in the origin country and with goverment effectiveness of the origin contry. From the plots below we can see that these reasons and the number of refugees have a correlation, with the drop of GDP, drop of controll of corruption (the raise of corruption) and the government effectivenes on the original contry the number of refugees is big from these origin contries.

In [ ]:
dataset = pd.read_csv('./2_dataset/final_dataset_preprocesed_v6.csv')

In [ ]:
sns.relplot(x="GDP_origin", y="Refugees under UNHCR’s mandate", data=dataset)
plt.savefig("./4_data_analysing_visualization/question_1_visualisation/NumberOfRefugees_GDP_Origin.JPG")

In [ ]:
sns.relplot(x="Control of Corruption_origin", y="Refugees under UNHCR’s mandate", data=dataset)
plt.savefig("./4_data_analysing_visualization/question_1_visualisation/NumberOfRefugees_CorruptionInOrigin.JPG")

In [ ]:
sns.relplot(x="Government Effectiveness_origin", y="Refugees under UNHCR’s mandate", data=dataset)
plt.savefig("./4_data_analysing_visualization/question_1_visualisation/NumberOfRefugees_GovernmentEffectiveness.JPG")

- 2. Do some events (i.e. war) initiate a higher amount of refugees than others (i.e. natural disasters, economic breakdown)?

We found out that some events create a higher number of refugees than others, but we can have also exceptions. Conflicts like wars, civil wars, internal conflicts create extremely more refugees flow than other events. Indexes like control of corruption, government effectiveness are also triggering but they create a smaller number of refugees and are spread over years. Another trigger is the economical reason which in some cases causes an extreme refugee flow (ex. Venezuela).
We can see from plots below that Syria in 2012 has an exponential increase in refugees flows because of the beginning of the war and Venezuela which has an extreme increase in refugees in the year 2018 even though there was no war, but the causes were economic reasons. We can see two similar increases in refugee flows, but the causes are different. The red bar indicates that country was in conflict in that year and the blue indicates the country was at peace.


In [ ]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('./2_dataset/final_dataset_preprocesed_v6.csv')
df = df.fillna(0)
df = df.round(2)

# Get People on /100000
# Information about country
# Country Parameter(Corruption etc) min and max value indexes

df_information = df[[
    'GDP_origin',
    'Control of Corruption_origin',
    'Government Effectiveness_origin',
    'Political Stability and Absence of Violence/Terrorism_origin',
    'Regulatory Quality_origin',
    'Rule of Law_origin',
    'Voice and Accountability_origin'
]]
# GDP_origin: min: 1.233485e+07 , max: 2.058016e+13
# Control of Corruption_origin: min: -1.87, max: 2.47
# Government Effectiveness_origin: min: -2.48, max: 2.44
# Political Stability and Absence of Violence/Terrorism_origin: min: -3.31, max: 1.76
# Regulatory Quality_origin: min: -2.65, max: 2.26
# Rule of Law_origin: min: -2.61, max: 2.10
# Voice and Accountability_origin: min: -2.31 ,  max: 1.80       


In [ ]:
#Group by data to Format for a country during Years
countries = df.groupby(["Country of origin", "Year"]).agg(
    {
     'Refugees under UNHCR’s mandate': 'sum',
     'Control of Corruption_origin':'min',
     'Political Stability and Absence of Violence/Terrorism_origin':'min',
     'Rule of Law_origin':'min',
     'Voice and Accountability_origin':'min',
     'Natural Deaths in origin': 'max',
     'Conflict in origin': 'max',
    })

countries

In [ ]:
#Group by data to Format for a country during Years
countries_corr = df.groupby(["Country of origin", "Year"]).agg(
    {
     'Refugees under UNHCR’s mandate': 'sum',
     'Control of Corruption_origin':'min',
     'Political Stability and Absence of Violence/Terrorism_origin':'min',
     'Rule of Law_origin':'min',
     'Voice and Accountability_origin':'min',
     'Natural Deaths in origin': 'max',
     'Conflict in origin': 'max',
     'GDP_origin': 'max' 
    })

countries_corr

# Mean attributes for each state during years
mean_correlation = countries_corr.reset_index()
mean_correlation.rename({'index': 'Country of origin'}, axis='columns', inplace=True)
#Get each state mean for each paramaters throw years 1996-2018
mean_correlation = countries_corr.groupby(["Country of origin"]).mean()
mean_correlation.corr()

In [ ]:
def visualizeCountryData(c, showTable = False):
    
    print(c)
    ctr =countries.loc[[c]]
    
    #   Display all Counntry data by years
    if showTable:
        countryData = ctr.T
        display(HTML(countryData.to_html()))

    
    ctr = ctr.reset_index()
    ctr.rename({'index': 'Country of origin'}, axis='columns', inplace=True)
    
#     display(HTML(ctr.corr().to_html()))

    
    colors = []
    for value in ctr.iterrows():
        if 1 == value[1]['Conflict in origin']:
            colors.append('r')
        else:
            colors.append('b')
    
  
    plt.bar(ctr['Year'], ctr['Refugees under UNHCR’s mandate'], align='center', alpha=0.5, color=colors)
    plt.ylabel('Refugees')
    plt.title(c)
    
    plt.xticks([1996,1999,2002,2005,2008,2011,2014,2018])
    
    
# #   Corruption Index
    minCorruptionIndex = ctr[ctr['Control of Corruption_origin'] == ctr['Control of Corruption_origin'].min()].Year
    
    if isinstance(minCorruptionIndex, pd.Series):
        minCorruptionIndex = minCorruptionIndex.values[0]
    
    plt.vlines(minCorruptionIndex, 0, ctr['Refugees under UNHCR’s mandate'].max(), linestyles ="dotted", colors ="r")
    plt.text(minCorruptionIndex,ctr['Refugees under UNHCR’s mandate'].max(),'Corruption: ' + str(ctr['Control of Corruption_origin'].min()))

    #   Natural Deaths in origin
    naturalDeathCases = ctr[ctr['Natural Deaths in origin'] == ctr['Natural Deaths in origin'].max()].Year

    if isinstance(naturalDeathCases, pd.Series):
        naturalDeathCases = naturalDeathCases.values[0] 
        
    if  naturalDeathCases.max() != 0.0 or naturalDeathCases != 0.0:    
        plt.vlines(naturalDeathCases, 0, ctr['Refugees under UNHCR’s mandate'].median(), linestyles ="dotted", colors ="b")
        plt.text(naturalDeathCases,ctr['Refugees under UNHCR’s mandate'].median(),'Natural D.: ' + str(ctr['Natural Deaths in origin'].max()) )
 
    
    #   Political Stability and Absence of Violence/Terrorism_origin
    ruleOfLawIndex = ctr[ctr['Rule of Law_origin'] == ctr['Rule of Law_origin'].min()].Year

    if isinstance(ruleOfLawIndex, pd.Series):
        ruleOfLawIndex = ruleOfLawIndex.values[0] 
        
    plt.vlines(ruleOfLawIndex, 0, ctr['Refugees under UNHCR’s mandate'].mean(), linestyles ="dotted", colors ="g")
    plt.text(ruleOfLawIndex,ctr['Refugees under UNHCR’s mandate'].mean() - 2,'R. of Law: ' + str(ctr['Rule of Law_origin'].min()))
    
    plt.savefig("./4_data_analysing_visualization/question_2_visualisation/"+c+".jpg")
    plt.show()

In [ ]:
visualizeCountryData('Ukraine', True)
visualizeCountryData('Venezuela (Bolivarian Republic of)', True)
visualizeCountryData('Syrian Arab Rep.', True)


- 3. What is the connection between origin and destination country?

From the plots below we can see that the number of refugees presented taking into consideration the migration destinations but under the condition of the distance in between origin contry and asylum contry is bigger when the distance is small leading to conclusion that the number of migration from the destination country is quite bigger in the neighbourhood countries. Also a connection in made in this matter taking into consideration the number of refugees and the GDP giving us the conclusion that the migration hattpens in matter of better GDP countries.
Both the cases where taken also for analysation just for the ballkan contries as a special case. 

In [ ]:
k_df = dataset[dataset["Country of origin"] == "Albania"]

In [ ]:
sns.relplot(y="Refugees under UNHCR’s mandate", x="Distance", hue="Country of origin", style="Country of origin",
            kind="line", data=k_df)
plt.savefig("./4_data_analysing_visualization/question_3_visualisation/OriginAndDestinationDistance_Ballkan.JPG")

In [ ]:
sns.relplot(y="Refugees under UNHCR’s mandate", x="Distance", hue="Country of origin",
            kind="line", data=dataset)
plt.savefig("./4_data_analysing_visualization/question_3_visualisation/OriginAndDestinationDistance.JPG")

In [ ]:
sns.relplot(y="Refugees under UNHCR’s mandate", x="GDP_asylum", hue="Country of origin", style="Country of origin",
            kind="line", data=k_df)
plt.savefig("./4_data_analysing_visualization/question_3_visualisation/OriginAndDestinationDistance_Ballkan_GDP.JPG")

In [ ]:
sns.relplot(y="Refugees under UNHCR’s mandate", x="GDP_asylum", hue="Country of origin", style="Country of origin",
            kind="line", data=dataset)
plt.savefig("./4_data_analysing_visualization/question_3_visualisation/OriginAndDestinationDistance_Ballkan__All_GDP.JPG")

### Prediction Part

Data preparation

In [ ]:
refugee_df = pd.read_csv('./2_dataset/final_dataset_preprocesed_v6.csv', sep=',')

In [ ]:
ml_df = refugee_df.drop(['Country of origin',
                         'Country of origin (ISO)', 
                         'Country of asylum', 
                         'Country of asylum (ISO)'], axis=1)


origin_cols = ['Control of Corruption_origin',
               'Government Effectiveness_origin',
               'Political Stability and Absence of Violence/Terrorism_origin',
               'Regulatory Quality_origin', 
               'Rule of Law_origin',
               'Voice and Accountability_origin']

asylum_cols = ['Control of Corruption_asylum',
               'Government Effectiveness_asylum',
               'Political Stability and Absence of Violence/Terrorism_asylum',
               'Regulatory Quality_asylum', 
               'Rule of Law_asylum',
               'Voice and Accountability_asylum']

for column in origin_cols:
    for ix in ml_df[ml_df[column].isna()].index:
        ml_df.loc[ix,'isna_statistics_origin'] = 1
        ml_df.loc[ix, column] = 0
        
for column in asylum_cols:
    for ix in ml_df[ml_df[column].isna()].index:
        ml_df.loc[ix,'isna_statistics_asylum'] = 1
        ml_df.loc[ix, column] = 0

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16, 16))
fig.tight_layout(pad=7.0)

pearsoncorr = ml_df.corr(method='pearson')
plot = sns.heatmap(pearsoncorr,
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5, 
            ax=ax)
plot.set_xticklabels(plot.get_xticklabels(), rotation=20, horizontalalignment='right')

### Prediction on the distance based on the origin country

The aim of this prediction attempt is to evaluate, whether it is possible to predict the distance between the country of origin and the country of asylum.

Based on the result scores, we deem the question to be NOT predictable

In [ ]:
tmp_df = ml_df.drop(['GDP_asylum', 
                     'Control of Corruption_asylum',
                     'Government Effectiveness_asylum',       
                     'Political Stability and Absence of Violence/Terrorism_asylum',
                     'Regulatory Quality_asylum', 
                     'Rule of Law_asylum',
                     'Voice and Accountability_asylum', 
                     'isna_GDP_asylum',
                     'isna_statistics_asylum', 
                     'Natural Deaths in origin',
                     'Conflict in origin'], axis=1)

scalers = [
    ('standard_scaler', StandardScaler())
]
models = [
    ('Random Forest', RandomForestRegressor()),
    ('ridge', RidgeCV()), 
    ('lasso', LassoCV()), 
    ('bayesian', BayesianRidge()),
    ('nearest neighbor', KNeighborsRegressor(n_neighbors=4))
]
target_column = 'Distance'

best_model = {}
best_val = 0
alphas = [0.2, 0.25, 0.3]

for alpha in alphas:
    for scaler_name, scaler in scalers:
        for model_name, model in models:
            
            y = tmp_df.get([target_column])
            X = tmp_df.drop([target_column], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=alpha)

            pipeline = Pipeline([
                ('scaler', scaler),
                ('clf', model)
            ])

            pipeline.fit(X_train, y_train.values.ravel())
            if model_name == 'linearregression':
                score = cross_val_score(pipeline, X_test, y_test, cv=10, scoring='r2')
                score = abs(score.mean())
            elif model_name == 'nearest neighbor':
                scores = {}
                for i in range(1, 6):
                    model.n_neighbors = i
                    pipeline = Pipeline([
                        ('scaler', scaler),
                        ('clf', model)
                    ])
                    
                    pipeline.fit(X_train, y_train.values.ravel())
                    
                    score = pipeline.score(X_test, y_test)
                    scores[score] = i
                    
                max_val = max(scores.keys())
                model.n_neighbors = scores[max_val]
                model_name += ' {}'.format(i)
                scores = max_val
            else:
                score = pipeline.score(X_test, y_test)

            if score > best_val:
                best_model['model'] = ((model_name, model))
                best_model['scaler'] = ((scaler_name, scaler))
                best_model['alpha'] = alpha
                best_val = score

print('Best Model: {} -> ({},{},{})'.format(
    best_val, 
    best_model['model'][0], 
    best_model['scaler'][0],
    best_model['alpha']
))

### Prediction on the amount of refugees between the country of origin and asylum

The aim of this prediction attempt was to determine whether ML algorithms are able to predict the amount of refugee which will be requesting asylum.

In [ ]:
tmp_df = ml_df.copy()

scalers = [
    ('standard_scaler', StandardScaler())
]
models = [
    ('ridge', RidgeCV()), 
    ('lasso', LassoCV()), 
    ('bayesian', BayesianRidge()),
    ('nearest neighbor', KNeighborsRegressor(n_neighbors=4)),
    ('Random Forest', RandomForestRegressor())
]
target_column = 'Refugees under UNHCR’s mandate'

best_model = {}
best_val = 0
alphas = [0.2, 0.25, 0.3]

for alpha in alphas:
    for scaler_name, scaler in scalers:
        for model_name, model in models:
            
            y = tmp_df.get([target_column])
            X = tmp_df.drop([target_column], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=alpha)

            pipeline = Pipeline([
                ('scaler', scaler),
                ('clf', model)
            ])

            pipeline.fit(X_train, y_train.values.ravel())
            if model_name == 'linearregression':
                score = cross_val_score(pipeline, X_test, y_test, cv=10, scoring='r2')
                score = abs(score.mean())
            elif model_name == 'nearest neighbor':
                scores = {}
                for i in range(1, 6):
                    model.n_neighbors = i
                    pipeline = Pipeline([
                        ('scaler', scaler),
                        ('clf', model)
                    ])
                    
                    pipeline.fit(X_train, y_train.values.ravel())
                    
                    score = pipeline.score(X_test, y_test)
                    scores[score] = i
                    
                max_val = max(scores.keys())
                model.n_neighbors = scores[max_val]
                model_name += ' {}'.format(i)
                scores = max_val
            else:
                score = pipeline.score(X_test, y_test)

            if score > best_val:
                best_model['model'] = ((model_name, model))
                best_model['scaler'] = ((scaler_name, scaler))
                best_model['alpha'] = alpha
                best_val = score

print('Best Model: {} -> ({},{},{})'.format(
    best_val, 
    best_model['model'][0], 
    best_model['scaler'][0],
    best_model['alpha']
))

### Predict the amount of refugees for neighboring countries

In [ ]:
tmp_df = ml_df[ml_df['Distance'] == 0].copy()

scalers = [
    ('standard_scaler', StandardScaler())
]
models = [
    ('ridge', RidgeCV()), 
    ('lasso', LassoCV()), 
    ('bayesian', BayesianRidge()),
    ('nearest neighbor', KNeighborsRegressor(n_neighbors=4)),
    ('Random Forest', RandomForestRegressor())
]
target_column = 'Refugees under UNHCR’s mandate'

best_model = {}
best_val = 0
alphas = [0.2, 0.25, 0.3]

for alpha in alphas:
    for scaler_name, scaler in scalers:
        for model_name, model in models:
            
            y = tmp_df.get([target_column])
            X = tmp_df.drop([target_column], axis=1)
            X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=alpha)

            pipeline = Pipeline([
                ('scaler', scaler),
                ('clf', model)
            ])

            pipeline.fit(X_train, y_train.values.ravel())
            if model_name == 'linearregression':
                score = cross_val_score(pipeline, X_test, y_test, cv=10, scoring='r2')
                score = abs(score.mean())
            elif model_name == 'nearest neighbor':
                scores = {}
                for i in range(1, 6):
                    model.n_neighbors = i
                    pipeline = Pipeline([
                        ('scaler', scaler),
                        ('clf', model)
                    ])
                    
                    pipeline.fit(X_train, y_train.values.ravel())
                    
                    score = pipeline.score(X_test, y_test)
                    scores[score] = i
                    
                max_val = max(scores.keys())
                model.n_neighbors = scores[max_val]
                model_name += ' {}'.format(i)
                scores = max_val
            else:
                score = pipeline.score(X_test, y_test)

            if score > best_val:
                best_model['model'] = ((model_name, model))
                best_model['scaler'] = ((scaler_name, scaler))
                best_model['alpha'] = alpha
                best_val = score

print('Best Model: {} -> ({},{},{})'.format(
    best_val, 
    best_model['model'][0], 
    best_model['scaler'][0],
    best_model['alpha']
))

### Summary

The 3 ML approaches are basic attempts at indicating a certain predictability between a country of origin and the country of asylum. Since the dataset from the dataanalysis part only contains 

country_of_origin -> distance, refugees, ... -> country_of_asylum

conntections, the good prediction results are not surprising.

The next 2 steps would be the following:


1. Extend the dataset to a country_of_origin -> distance, refugees -> for all country in countries layout

This would enable proper prediction results

2. Extend the dataset with more attributes about the country of asylum. The reason is that possible countries of asylum which are thousand kilometers away depend on a lot more characteristics than neighboring countries. Best example: What made Syrian refugees travel to Germany?

##  Were there any difficulties in analysing the data? What were the key insights obtained?

In the regard of analyzing data there are many difficulties that make this process really hard. During this exercise we encountered many of these difficulties in real life scenarios. In this exercise we had to deal with data from different datasets, since these datasets weren’t constructed for the same purpose difficulties arise in merging them. After merging these datasets, we ended with a lot of missing values which pointed it out the question, do we have in our dataset meaningful data. Another difficult aspect is to be able to point out relations between different features by using appropriate visualizations techniques. The insights obtained in regard of visualization where that in order to be able to produce visualizations that provide meaning it is best to start by investigating correlation between features this could be done by using correlation matrixes. After visualization the correlation matrix we can start to plot different features by comparing them in visualization. Having plots of the features that are correlated with each other would allow us to investigate different patterns in the data and maybe other hypothesis could be created by just looking at visualizations.

## Which Data Science tools and techniques were learned during this exercise?

During this exercise we had the oportunity to learn different techniques in data science such as different visualisation with seaborn library, missing values with K-NN handling, different merging techniques, models and evaluation of the models.